# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Read in file 
school_file = "Resources/schools_complete.csv"
student_file = "Resources/students_complete.csv"

# Store imported files as new DFs
school_df = pd.read_csv(school_file)
student_df = pd.read_csv(student_file)

## Print statements are included for checking purposes - can be deleted if neccessary 
## Print rows of each DF:
# school_df
# student_df

In [2]:
# Rename col "school_name" in school_df to "school name"
school_df.rename({"school_name" : "school name"}, axis=1, inplace=True)

# Make a copy of school_df to manipulate and reuse later throughout the code
# Store renamed school_df as new df
copy_school_sum = school_df.copy()


## Print statements are included for checking purposes - can be deleted if neccessary 
## Print both DFs to verify that the columns are the same
#school_df.columns
#copy_school_sum.columns

In [3]:
# Rename col "school" in student_df to "school name"
# Store renamed student_df as new df
renamed_student_df = student_df.rename(columns={"school_name" : "school name"})


## Print statements are included for checking purposes - can be deleted if neccessary 
## Print both DFs to verify that the columns are the same
#renamed_student_df.columns

In [4]:
## Total Schools
total_schools = school_df["school name"].count()
#print(total_schools)


## Total Students
total_students = renamed_student_df["school name"].count()
#print(total_students)

## Total Budget
total_budget = school_df["budget"].sum()

## Average Math Score
avg_math_score = renamed_student_df["math_score"].mean()

## Average Reading Score
avg_read_score = renamed_student_df["reading_score"].mean()

In [5]:
## % Passing Math based on 70
math_pass = renamed_student_df.loc[(student_df["math_score"] >= 70)]

count_pass_math = math_pass["math_score"].count()
#print(count_pass_math)

per_math_pass = (count_pass_math/total_students)*100
#print(per_math_pass)

In [6]:
## % Passing Reading based on 70
read_pass = renamed_student_df.loc[(student_df["reading_score"] >= 70)]

count_pass_read = read_pass["reading_score"].count()
#print(count_pass_read)

per_read_pass = (count_pass_read/total_students)*100
#print(per_read_pass)

In [7]:
## Overall Passing Rate (Average of the above two)
overall_pass = (per_math_pass + per_read_pass)/2
#print(overall_pass)

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [8]:
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                                   "Total Students": [total_students],
                                   "Total Budget": [total_budget],
                                   "Average Math Score": [avg_math_score],
                                   "Average Reading Score": [avg_read_score],
                                   "% Passing Math":[per_math_pass],
                                   "% Passing Reading":[per_read_pass],
                                   "% Overall Passing Rate": [overall_pass]})
district_summary['Total Budget'] = district_summary['Total Budget'].map('${:,.2f}'.format)
district_summary=district_summary[['Total Schools','Total Students','Total Budget','Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]

district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


In [9]:
# First, we will delete the School ID col from copy_school_sum because we will not need it for future use
del copy_school_sum['School ID']

# We can print copy_school_sum to verify the del
# copy_school_sum

In [10]:
## Calculate the Per Student Budget
## We will pull the school budget and divide it by the school size to determine the budget per student
## Then, we will create a new col named 'Per Student Budget'

copy_school_sum['Per Student Budget'] = copy_school_sum['budget']/copy_school_sum['size']


# copy_school_sum

In [11]:
## Average Math Score & Average Reading Score
## We will use a groupby function to group on school name and display both reading and math scores from above
## then reset the index

avg_math_read_tbl = renamed_student_df.groupby(['school name'])['reading_score', 'math_score'].mean().reset_index()

# avg_math_read_tbl

In [12]:
## Now, we merge avg_math_read_tbl with copy_school_sum df
## We want to merge on School Name and by 'outer' to include everything

copy_school_sum = copy_school_sum.merge(avg_math_read_tbl, on='school name', how="outer")

#copy_school_sum

In [13]:
## We will use conditionals to find the % Passing Math and % Passing Reading
## Then, we will store them as new variabels
## We are using a passrate of 70% or higher

# % Passing Reading
summary_passing_read = renamed_student_df[renamed_student_df['reading_score']>=70]

#% Passing Math
summary_passing_math = renamed_student_df[renamed_student_df['math_score']>=70]

#summary_passing_read
#summary_passing_math

In [14]:
## Count the number of students passing in reading 
pass_read_count_sum = summary_passing_read.groupby(["school name"])['reading_score'].count().reset_index()

## Then, rename the column the 'reading_score' to 'Reading Count'
pass_read_count_sum.rename({'reading_score' : 'Reading Count'}, axis=1, inplace=True)

#pass_read_count_sum

In [15]:
## Count the number of students passing in math 
pass_math_count_sum = summary_passing_math.groupby(["school name"])['math_score'].count().reset_index()

## Then, rename the column the 'math_score' to 'Math Count'
pass_math_count_sum.rename({'math_score' : 'Math Count'}, axis=1, inplace=True)

#pass_math_count_sum

In [16]:
## Merge pass_math_count_sum with pass_read_count_sum
## We want to merge on School Name and by 'inner' to include only the contents found in both columns

pass_count = pass_math_count_sum.merge(pass_read_count_sum, on="school name", how='inner')

#pass_count

In [17]:
## Merge copy_school_sum with pass_count we just created
## We want to merge on School Name and by 'outer' to include everything

copy_school_sum = copy_school_sum.merge(pass_count, on="school name", how='outer')

#copy_school_sum

In [18]:
## Calc % passing math and reading
## Take the subject count and divide by the school size, then pmultiply by 100 to get percentage

# % Passing Math
copy_school_sum['% Passing Math'] = (copy_school_sum['Math Count']/copy_school_sum['size'])*100

# % Passing Reading
copy_school_sum['% Passing Reading'] = (copy_school_sum['Reading Count']/copy_school_sum['size'])*100

#copy_school_sum

In [19]:
# Now, we will delete the Math Count and Reading Count cols from copy_school_sum 
# because we will not need the counts for future use

del copy_school_sum['Math Count']
del copy_school_sum['Reading Count']

#copy_school_sum

In [20]:
## Calc % overall passing 
## Overall Passing Rate (Average of the above two)

copy_school_sum['% Overall Passing'] = (copy_school_sum['% Passing Math'] + copy_school_sum['% Passing Reading'])/2

#copy_school_sum

In [21]:
# now, rename axis for reading and math scores to Avg. Reading Score and Avg. Math Score in copy_school_sum
copy_school_sum.rename({'reading_score':'Avg. Reading Score',
                             'math_score':'Avg. Math Score'}, axis=1, inplace=True)



## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [22]:
copy_school_sum

,school name,type,size,budget,Per Student Budget,Avg. Reading Score,Avg. Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,95.586652
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.680064,81.933280,74.306672
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.505855,96.252927,94.379391
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [23]:
## Create a table that highlights the top 5 performing schools based on Overall Passing Rate. 
## Found the top 5 performing by sorting copy_school_sum on the '% Overall Passing' col in copy_school_sum df 
## By sorting, we can find the five top performing

top_performing_by_pr_df = copy_school_sum.sort_values(by=['% Overall Passing'], ascending=False).head(5)

In [24]:
top_performing_by_pr_df.head(5)

,school name,type,size,budget,Per Student Budget,Avg. Reading Score,Avg. Math Score,% Passing Math,% Passing Reading,% Overall Passing
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [25]:
## Create a table that highlights the 5 worst performing schools based on Overall Passing Rate. 
## Include all of the same metrics as above.
## Using the copy_school_sum df found the bottom 5 performing schools based on Overall Passing Rate col
worst_performing_by_pr_df = copy_school_sum.sort_values(by=['% Overall Passing']).head(5)

In [26]:
worst_performing_by_pr_df

,school name,type,size,budget,Per Student Budget,Avg. Reading Score,Avg. Math Score,% Passing Math,% Passing Reading,% Overall Passing
11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,73.363852
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,73.500171
12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,66.057551,81.222432,73.639992
13,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [27]:
#creates grade level average math scores for each school 
ninth_math = student_df.loc[student_df['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = student_df.loc[student_df['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = student_df.loc[student_df['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = student_df.loc[student_df['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})

math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "school_name"

#show and format
math_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [28]:
#creates grade level average reading scores for each school
ninth_reading = student_df.loc[student_df['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_reading = student_df.loc[student_df['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_reading = student_df.loc[student_df['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_reading = student_df.loc[student_df['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

#merges the reading score averages by school and grade together
reading_scores = pd.DataFrame({
        "9th": ninth_reading,
        "10th": tenth_reading,
        "11th": eleventh_reading,
        "12th": twelfth_reading
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "school_name"

#format
reading_scores.style.format({'9th': '{:.1f}', 
                             "10th": '{:.1f}', 
                             "11th": "{:.1f}", 
                             "12th": "{:.1f}"})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [29]:
# create average spend bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
copy_school_sum["Spending Ranges (Per Student)"] = pd.cut(copy_school_sum["Per Student Budget"], spending_bins, labels=group_names)

school_spending_grouped = copy_school_sum.groupby("Spending Ranges (Per Student)").mean()

#Removing the unwanted columns per the sample provided
del school_spending_grouped['size']
del school_spending_grouped['budget']
del school_spending_grouped['Per Student Budget']


school_spending_grouped

          


,Avg. Reading Score,Avg. Math Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.933814,83.455399,93.460096,96.610877,95.035486
$585-615,83.885211,83.599686,94.230858,95.900287,95.065572
$615-645,81.891436,79.079225,75.668212,86.106569,80.887391
$645-675,81.027843,76.997210,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [31]:
#create school size bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
copy_school_sum["School Size"] = pd.cut(copy_school_sum["size"], size_bins, labels=group_names)

school_size_grouped = copy_school_sum.groupby("School Size").mean()

#Removing the unwanted columns per the sample provided
del school_size_grouped['budget']
del school_size_grouped['Per Student Budget']
del school_size_grouped['size']

school_size_grouped


,Avg. Reading Score,Avg. Math Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.929843,83.821598,93.550225,96.099437,94.824831
Medium (1000-2000),83.864438,83.374684,93.599695,96.790680,95.195187
Large (2000-5000),81.344493,77.746417,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [42]:

copy_copy_school_sum = copy_school_sum
copy_copy_school_sum
#Scores by School Type
schools_score_type = copy_copy_school_sum.groupby(['type'])[["type",
                                                             "Avg. Math Score",
                                                             "Avg. Reading Score",
                                                             "% Passing Math",
                                                             "% Passing Reading",
                                                             "% Overall Passing"]].mean()
schools_score_type.head()
#Renaming columns in dataframe



,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [ ]:
#Obversations:
#1 School size and score results are correlated. The higher the number of students, the lower success in grades.
#2 Based on scores by school type, charter schools have higher pass rates in reading and math than district schools. This is validated by the top five performing schools all being charter schools.
#3 Comparing student’s math scores and reading scores by grade, they have lower math than reading scores.
